In [1]:
# calcul matriciel
import numpy as np
# utilisation des modèles
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

2024-07-02 12:11:40.290629: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-02 12:11:40.292526: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 12:11:40.330246: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 12:11:40.452253: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 12:11:41.295297: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
class MNISTLoader:
  def __init__(self):
    self.x_train = None
    self.x_test = None
    self.x_val = None
    
    self.x_test_normalized = None
    self.x_train_normalized_80 = None
    self.x_val_normalized_20 = None
    self.y_test = None
    self.y_train_80 = None
    self.y_val_20 = None


  def load_data(self):
    (x_train, y_train), (self.x_test, self.y_test) = tf.keras.datasets.mnist.load_data()
    x_train_normalized = x_train / 255
    self.x_test_normalized = self.x_test / 255

    self.x_train, self.x_val, self.y_train_80, self.y_val_20 = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
    self.x_train_normalized_80, self.x_val_normalized_20, self.y_train_80, self.y_val_20 = train_test_split(x_train_normalized, y_train, test_size=0.2, random_state=42)


loader = MNISTLoader()
loader.load_data()


# Convolutional model with handly data

In [6]:
num_classes = 10
core_size = 4
model = Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(128, core_size, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, core_size, activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(loader.x_train_normalized_80,
          loader.y_train_80,
          validation_data=(loader.x_val_normalized_20, loader.y_val_20),
          epochs=10)

model.evaluate(loader.x_test_normalized, loader.y_test)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.9035 - loss: 0.3126 - val_accuracy: 0.9856 - val_loss: 0.0486
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.9864 - loss: 0.0449 - val_accuracy: 0.9833 - val_loss: 0.0501
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.9898 - loss: 0.0317 - val_accuracy: 0.9889 - val_loss: 0.0385
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9937 - loss: 0.0199 - val_accuracy: 0.9889 - val_loss: 0.0393
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9938 - loss: 0.0175 - val_accuracy: 0.9912 - val_loss: 0.0326
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9959 - loss: 0.0136 - val_accuracy: 0.9902 - val_loss: 0.0379
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9970 - loss: 0.0099 - val_accuracy: 0.9877 - val_loss: 0.0446
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9972 - l

[0.03459934517741203, 0.9911999702453613]

In [8]:
import tkinter as tk
from PIL import Image, ImageDraw

# Création de la fenêtre principale
window = tk.Tk()

canvas = tk.Canvas(window, width=280, height=280, bg='white')
canvas.pack()

image = Image.new('L', (280, 280), 255)
draw = ImageDraw.Draw(image)

def clear_canvas():
    canvas.delete('all')
    global image, draw
    image = Image.new('L', (280, 280), 255)
    draw = ImageDraw.Draw(image)

clear_button = tk.Button(window, text='Clear', command=clear_canvas)
clear_button.pack()


# Fonction appelée lors du dessin
def draw_on_canvas(event):
    x, y = event.x, event.y
    canvas.create_oval(x, y, x+10, y+10, fill='black')
    draw.rectangle([x, y, x+10, y+10], fill='black')

# Fonction appelée lors de la prédiction
def predict_digit():
    # Redimensionner l'image à la taille attendue par le modèle (28x28)
    resized_image = image.resize((28, 28))

    # Prétraitement de l'image pour l'adapter au modèle (conversion en tableau numpy, normalisation, etc.)
    preprocessed_image = preprocess_image(resized_image)

    # Effectuer la prédiction avec le modèle
    predictions = model.predict(np.expand_dims([preprocessed_image], axis=-1))[0]
    prediction = np.argmax(predictions)

    # Afficher la prédiction
    prediction_label.config(text='Prediction: ' + str(prediction))

# Fonction pour prétraiter l'image avant la prédiction
def preprocess_image(image):
    # Convertir l'image en niveaux de gris
    image = image.convert('L')

    # Convertir l'image en tableau numpy
    image_array = np.array(image)

    # Inverser le noir et le blanc
    image_array = 255 - image_array

    # Trouver les limites du dessin (les coordonnées où il y a du noir)
    nonzero_indices = np.nonzero(image_array)
    min_x = np.min(nonzero_indices[1])
    max_x = np.max(nonzero_indices[1])
    min_y = np.min(nonzero_indices[0])
    max_y = np.max(nonzero_indices[0])

    # Extraire la zone contenant le dessin et la centrer dans une image 28x28
    cropped_image = image_array[min_y:max_y, min_x:max_x]

    # Redimensionner l'image à 28x28
    resized_image = Image.fromarray(cropped_image).resize((28, 28))

    # Normaliser l'image comme dans votre chargement de données
    preprocessed_image = np.array(resized_image) / 255.0

    # Ajouter une dimension pour correspondre à la forme attendue par le modèle
    preprocessed_image = np.expand_dims(preprocessed_image, axis=-1)

    return preprocessed_image

predict_button = tk.Button(window, text='Predict', command=predict_digit)
predict_button.pack()

prediction_label = tk.Label(window, text='Prediction: ')
prediction_label.pack()

# Capturer les événements de dessin sur la toile
canvas.bind('<B1-Motion>', draw_on_canvas)

# Lancer l'application
window.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
